In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# !kill -9 -1 or !pkill -9 -f ipykernel_launcher

E: Package 'python-software-properties' has no installation candidate


In [1]:
import tensorflow as tf
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train  /= 255
x_test /= 255

Using TensorFlow backend.


170500096/170498071 [==============================] - 41s 0us/step


In [0]:
vgg = tf.keras.models.Sequential()

# Layer 1
vgg.add(tf.layers.Conv2D(64, kernel_size=(3, 3), padding='same', input_shape = x_train.shape[1:], activation=tf.nn.relu))
vgg.add(tf.keras.layers.BatchNormalization())
vgg.add(tf.layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation=tf.nn.relu))
vgg.add(tf.keras.layers.BatchNormalization())
vgg.add(tf.layers.MaxPooling2D(pool_size=(2, 2), strides=2))

# Layer 2
vgg.add(tf.layers.Conv2D(128, kernel_size=(3, 3), padding='same', activation=tf.nn.relu))
vgg.add(tf.keras.layers.BatchNormalization())
vgg.add(tf.layers.Conv2D(128, kernel_size=(3, 3), padding='same', activation=tf.nn.relu))
vgg.add(tf.keras.layers.BatchNormalization())
vgg.add(tf.layers.MaxPooling2D(pool_size=(2, 2), strides=2))

# Layer 3
vgg.add(tf.layers.Conv2D(256, kernel_size=(3, 3), padding='same', activation=tf.nn.relu))
vgg.add(tf.keras.layers.BatchNormalization())
vgg.add(tf.layers.Conv2D(256, kernel_size=(3, 3), padding='same', activation=tf.nn.relu))
vgg.add(tf.keras.layers.BatchNormalization())
vgg.add(tf.layers.MaxPooling2D(pool_size=(2, 2),  strides=2))

# Layer 4
vgg.add(tf.layers.Conv2D(512, kernel_size=(3, 3), padding='same', activation=tf.nn.relu))
vgg.add(tf.keras.layers.BatchNormalization())
vgg.add(tf.layers.Conv2D(512, kernel_size=(3, 3), padding='same', activation=tf.nn.relu))
vgg.add(tf.keras.layers.BatchNormalization())
vgg.add(tf.layers.MaxPooling2D(pool_size=(2, 2), strides=2))

# Layer 5
#vgg.add(tf.layers.Conv2D(512, kernel_size=(3, 3), padding='same', activation=tf.nn.relu))
#vgg.add(tf.keras.layers.BatchNormalization())
#vgg.add(tf.layers.Conv2D(512, kernel_size=(3, 3), padding='same', activation=tf.nn.relu))
#vgg.add(tf.keras.layers.BatchNormalization())
#vgg.add(tf.layers.MaxPooling2D(pool_size=(2, 2), strides=2))

# Layer 6
vgg.add(tf.layers.Flatten())
vgg.add(tf.layers.Dense(1024, activation=tf.nn.relu))
vgg.add(tf.keras.layers.BatchNormalization())
vgg.add(tf.layers.Dropout(0.5))
vgg.add(tf.layers.Dense(10, activation=tf.nn.softmax))

In [0]:
sgd = tf.keras.optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
vgg.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
datagen = ImageDataGenerator(horizontal_flip=True,
        width_shift_range=0.1, height_shift_range=0.1)

datagen.fit(x_train)

vgg.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
                    steps_per_epoch=x_train.shape[0] // 64,
                    epochs=60,
                    verbose=1,validation_data=(x_test,y_test))


vgg.evaluate(x_test, y_test)

Epoch 1/60
781/781 [==============================] - 62s 79ms/step - loss: 2.3920 - acc: 0.2829 - val_loss: 2.5885 - val_acc: 0.1623
Epoch 2/60
781/781 [==============================] - 56s 72ms/step - loss: 1.7248 - acc: 0.3745 - val_loss: 1.6080 - val_acc: 0.4083
Epoch 3/60
781/781 [==============================] - 56s 72ms/step - loss: 1.4744 - acc: 0.4632 - val_loss: 1.5274 - val_acc: 0.4692
Epoch 4/60
781/781 [==============================] - 56s 72ms/step - loss: 1.3370 - acc: 0.5199 - val_loss: 1.2933 - val_acc: 0.5392
Epoch 5/60
781/781 [==============================] - 56s 71ms/step - loss: 1.2182 - acc: 0.5649 - val_loss: 1.3360 - val_acc: 0.5545
Epoch 6/60
781/781 [==============================] - 56s 72ms/step - loss: 1.1144 - acc: 0.6022 - val_loss: 1.2263 - val_acc: 0.5946
Epoch 7/60
781/781 [==============================] - 56s 72ms/step - loss: 1.0080 - acc: 0.6416 - val_loss: 1.0300 - val_acc: 0.6704
Epoch 8/60
781/781 [==============================] - 56s 72ms

[0.6147665899574757, 0.8583]

In [11]:
# Buffered data was truncated after reaching the output size limit
scores = vgg.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

10000/10000 [==============================] - 3s 287us/step

Test result: 85.830 loss: 0.615
